[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [3]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"I need ice cream"
signature = private_key.sign(message)

In [2]:
public_key.verify(signature, message)

True

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [4]:
class Transfer:
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key

class ECDSACoin:
    def __init__(self, transfers):
        self.transfers = transfers

In [5]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [14]:
from utils import serialize

def issue(public_key):
    # Create a message specifying who the coin is being issued to
    message = serialize(public_key)
    
    # Create the first transfer, signing with the banks private key
    signature = bank_private_key.sign(message)
    transfer = Transfer(
        signature=signature,
        public_key=public_key,
    )
    
    # Create and return the coin with just the issuing transfer
    coin = ECDSACoin(transfers=[transfer])
    return coin

# Validating the First Transfer

In [19]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    assert bank_public_key.verify(transfer.signature, message)

In [20]:
# give alice a ECDSACoin
alice_coin = issue(alice_public_key)

validate(alice_coin)

In [27]:
# Example where the signature is done using the wrong private key
transfer = Transfer(
    signature=bob_private_key.sign(serialize(bob_public_key)),
    public_key=bob_public_key
)
bad_coin = ECDSACoin([transfer])

In [28]:
from ecdsa import BadSignatureError

try:
    validate(bad_coin)
except BadSignatureError:
    print("Bad Signature detected")

Bad Signature detected


# Validating Subsequent Transfers

In [43]:
def transfer_message(signature, public_key):
    return serialize({
        "previous_signature": signature,
        "next_owner_public_key": public_key
    })

def validate(coin):
    # Check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    assert bank_public_key.verify(transfer.signature, message)

    # Check the rest of coin.transfers
    previous_transfer = coin.transfers[0]
    for transfer in coin.transfers[1:]:
        # Check previous owner signed this transfer using their private key
        assert previous_transfer.public_key.verify(
            transfer.signature,
            transfer_message(previous_transfer.signarure, transfer.public_key)
        )
        previous_transfer = transfer
    return True

In [40]:
def get_owner(coin):
    database = {
        serialize(bob_public_key): "Bob",
        serialize(alice_public_key): "Alice",
        serialize(bank_public_key): "Bank",
    }
    public_key = serialize(coin.transfers[-1].public_key)
    return database[public_key]

In [46]:
coin = issue(alice_public_key)

print("This coin is owned by", get_owner(coin))

m1 = transfer_message(coin.transfers[-1].signature, bob_public_key)

alice_to_bob = Transfer(
    signature=alice_private_key.sign(m1),
    public_key=bob_public_key
)
coin.transfers.append(alice_to_bob)

print("This coin is owned by", get_owner(coin))

m2 = transfer_message(coin.transfers[-1].signature, bank_public_key)
bob_to_bank = Transfer(
    signature=bob_private_key.sign(m2),
    public_key=bank_public_key,
)
coin.transfers.append(bob_to_bank)

print("This coin is owned by", get_owner(coin))


This coin is owned by Alice
This coin is owned by Bob
This coin is owned by Bank


# Serialization

In [47]:
from utils import to_disk, from_disk

In [49]:
import os

filename = "serialized/coin.ecdsacoin"

print("Does the coinfile exists on disk?", os.path.isfile(filename))

Does the coinfile exists on disk? False


In [50]:
coin = issue(alice_public_key)

to_disk(coin, filename)

In [51]:
print("Does the coinfile exists on disk?", os.path.isfile(filename))

Does the coinfile exists on disk? True


In [52]:
coin = from_disk(filename)

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [1]:
from ecdsacoin import issue, transfer_message

coin = issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()

NameError: name 'ecdsacoin' is not defined